<img src="../../images/arm_logo.png" width=500 alt="ARM Logo"></img>

# Py-ART Gridding
---

This notebook shows how you can easily make Cartesian grids from radar files using Py-ART.

In [ ]:
import os
import warnings

import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np


import pyart
from pyart.testing import get_test_data

warnings.filterwarnings('ignore')

Let's load and display our radar file!

In [ ]:
file = 'data/swx_20120520_0641.nc'
radar = pyart.io.read(file)

In [ ]:
display = pyart.graph.RadarMapDisplay(radar)
fig = plt.figure(figsize=(8, 6))
ax1 = plt.subplot(1, 1, 1, projection=ccrs.PlateCarree())
display.plot_ppi_map('reflectivity_horizontal', ax=ax1, sweep=2, resolution='50m',
                     vmin=0, vmax=60, 
                     projection=ccrs.PlateCarree(), cmap='pyart_HomeyerRainbow')

Py-ART has the Grid object which has characteristics that are similar to that of the Radar object, except that the data are stored in Cartesian coordinates instead of the radar's coordinates. Much like the Radar object, the fields are accessible by simply accessing grid.fields['field_name']['data'], and the object contains parameters such as the radar's location and projection. Full information about the Grid object is available here:

https://arm-doe.github.io/pyart/API/generated/pyart.core.Grid.html

We can create a Grid object from a Radar object by using pyart.map.grid_from_radars()

In [ ]:
# create a basic gate filter which specifies gates to exclude from gridding.
gatefilter = pyart.filters.GateFilter(radar)
gatefilter.exclude_transition()
gatefilter.exclude_invalid('mean_doppler_velocity')
gatefilter.exclude_invalid('reflectivity_horizontal')
gatefilter.exclude_outside('reflectivity_horizontal', 0, 80)

In [ ]:
grid = pyart.map.grid_from_radars(
    [radar], grid_shape=(41, 101, 101), 
    grid_limits=((0., 20000,), (-40000, 40000.), (-40000, 40000.)),
    gatefilter=gatefilter)

The Display module contains routines for plotting gridded data in GridMapDisplay, which has functionality similar to RadarMapDisplay. The below example plots the Grid that was just made at the 1.5 km level. The level can be specified in the level parameter of plot_grid. Feel free to take the time to look at higher altitudes by changing this parameter.

In [ ]:
gdisplay = pyart.graph.GridMapDisplay(grid)
gdisplay.plot_grid('reflectivity_horizontal', level=3, cmap='pyart_HomeyerRainbow', vmin=-20, vmax=60)

We can even do both longitudnal and latitudnal slices. Change the lat parameter to view different portions of the storm! 

In [ ]:
gdisplay.plot_latitude_slice('reflectivity_horizontal', cmap='pyart_HomeyerRainbow', vmin=0, vmax=60)
plt.xlim([-30, 30])

As you can see, this Grid suffers from gaps in data as well as artificial "feathers" near the top that are simply the result of projecting the data to Cartesian coordinates. We would like to eliminate these as much as possible. Thankfully, Py-ART let's us, on top of specifying the grid resolution, also place in a custom radius of influence which controls how many points surrounding the gate are factored into the interpolation. Let's try increasing the minimum radius of influence to 1.75 km!

In [ ]:
grid_better = pyart.map.grid_from_radars(
    [radar], grid_shape=(41, 201, 201), 
    grid_limits=((0., 20000,), (-40000., 40000.), (-40000, 40000.)),
    roi_func='dist_beam', min_radius=1750.0)

As you can see, most of the artifacts at higher altitudes have been smoothed out, but we have lost some spatial resolution as a result.

In [ ]:
gdisplay2 = pyart.graph.GridMapDisplay(grid_better)
gdisplay2.plot_latitude_slice('reflectivity_horizontal', cmap='pyart_HomeyerRainbow', vmin=0, vmax=60)
plt.xlim([-30, 30])